In [ ]:
!pip install np_utils

     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-cp37-none-any.whl size=57133 sha256=5d61640cb65cd93c3bee19769cc0a814873cc4780b037546b9a5b856f8498d87
  Stored in directory: /root/.cache/pip/wheels/92/4b/81/206efd0d01330a96f3aebe5021d2d5f0b264b7ade827c306ef
Successfully built np-utils


In [ ]:
import os
os.listdir("drive/MyDrive/Music-Generator")

FileNotFoundError: ignored

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import collections
import glob
import json
import os
import pickle
import numpy

from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.metrics import *
try:
  import google.colab
  IS_ON_GOOGLE_COLAB = True
except:
  IS_ON_GOOGLE_COLAB = False

TEST_RUN = False
NORMALIZE_NOTES = True
NORMALIZATION_BOUNDARIES = [3, 4]

EPOCHS = 50
MODEL_DIR_PATH = "generated_models/"
MODEL_NAME = "cpc"
OCCURENCES = "data/occurences"
DATA_NOTES_DIR = "data/notes"
MIDI_SONGS_REGEX = "midi_songs/*.mid"
CHECKPOINTS = "checkpoints/"
LOGS = "logs/"


GOOGLE_COLAB_ROOT = "drive/MyDrive/Music-Generator/"

if IS_ON_GOOGLE_COLAB:
    MODEL_DIR_PATH = GOOGLE_COLAB_ROOT + MODEL_DIR_PATH
    DATA_NOTES_DIR = GOOGLE_COLAB_ROOT + DATA_NOTES_DIR
    MIDI_SONGS_REGEX = GOOGLE_COLAB_ROOT + MIDI_SONGS_REGEX
    OCCURENCES = GOOGLE_COLAB_ROOT + OCCURENCES
    CHECKPOINTS = GOOGLE_COLAB_ROOT
    LOGS = GOOGLE_COLAB_ROOT + LOGS


if TEST_RUN:
    EPOCHS = 100

def train_network(checkpoint_path=None):
    """ Train a Neural Network to generate music """
    notes = get_notes()
    curr_dt = get_current_datetime()
    print(str("Current datatime: " + curr_dt))
    # get amount of pitch names
    n_vocab = len(set(notes))
    print("Vocabulary size: " + str(n_vocab))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    if checkpoint_path:
      model.load_weights(checkpoint_path)

    train(model, network_input, network_output, curr_dt)

def normalize_note(pitch: note.Note):
    if pitch.octave not in range(NORMALIZATION_BOUNDARIES[0], NORMALIZATION_BOUNDARIES[1]):
        if pitch.octave < NORMALIZATION_BOUNDARIES[0]:
            pitch.octave = NORMALIZATION_BOUNDARIES[0]
        elif pitch.octave > NORMALIZATION_BOUNDARIES[1]:
            pitch.octave = NORMALIZATION_BOUNDARIES[1]
    return pitch

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    # max_i = 1
    for i, file in enumerate(glob.glob(MIDI_SONGS_REGEX)):
        if TEST_RUN and i == 1:
            break
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                if NORMALIZE_NOTES:
                    element = normalize_note(element)
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                # notes.append('.'.join(str(n) for n in element.normalOrder))
                if NORMALIZE_NOTES:
                    notes.append('.'.join(str(normalize_note(n)) for n in element.pitches))
                else:
                    notes.append('.'.join(str(n) for n in element.pitches))

    with open(DATA_NOTES_DIR, 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100


    # get all pitch names
    occurences = collections.defaultdict(int)
    pitchnames = sorted(set(item for item in notes))
    for note in notes:
        occurences[note] += 1
    # with open('data/occurences', 'wb') as filepath:
    #     pickle.dump(occurences, filepath)

    jsonStr = json.dumps(occurences)
    with open(OCCURENCES, 'w') as f:
        f.writelines(jsonStr)
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',  metrics=[CategoricalAccuracy()])
    return model

def get_current_datetime():
    from datetime import datetime
    now = datetime.now()
    dt_name = now.strftime("%m_%d_%Y__%H_%M_%S")
    return dt_name

def train(model, network_input, network_output, curr_dt):
    """ train the neural network """
    # filepath = CHECKPOINTS + "weights-improvement-{epoch:02d}-{loss:.4f}-{categorical_accuracy:.4f}-bigger.hdf5"
    # filepath = "weights-improvement-epoch:{epoch:02d}-loss:{loss:.4f}-cat_acc:{categorical_accuracy:.4f}.hdf5"
    filepath =  CHECKPOINTS + str(curr_dt) + "/" + "epoch:{epoch:02d}-loss:{loss:.4f}-cat_acc:{categorical_accuracy:.4f}.hdf5"

    # filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='categorical_accuracy',
        verbose=0,
        save_best_only=True,
        mode='max'
    )
    log = tf.keras.callbacks.TensorBoard(log_dir=LOGS + curr_dt),
     
    callbacks_list = [checkpoint, log]


    history = model.fit(network_input, network_output, epochs=EPOCHS, batch_size=128, callbacks=callbacks_list)
    print(history.history)
    print(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")
    model.save(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if __name__ == '__main__':
    train_network("/content/drive/MyDrive/Music-Generator/06_27_2021__09_18_22/epoch:27-loss:0.7527-cat_acc:0.8634.hdf5")

Parsing drive/MyDrive/Music-Generator/midi_songs/Gold_Silver_Rival_Battle.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Ff7-Cinco.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Rachel_Piano_tempofix.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Ff7-One_Winged.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Ff7-Jenova_Absolute.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/FF3_Battle_(Piano).mid
Parsing drive/MyDrive/Music-Generator/midi_songs/FF6epitaph_piano.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Fyw_piano.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing drive/MyDrive/Music-Generator/midi_songs/HighwindTakestotheSkies.mid
Parsing drive/MyDri

In [ ]:
if __name__ == '__main__':
    train_network()

In [ ]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.models import load_model

try:
  import google.colab
  IS_ON_GOOGLE_COLAB = True
except:
  IS_ON_GOOGLE_COLAB = False

GOOGLE_COLAB_ROOT = "drive/MyDrive/Music-Generator/"
GENERATED_MODEL_PATH = "generated_models/cpc_06_25_2021__09_42_21.hdf5"
DATA_NOTES_DIR = "data/notes"

if IS_ON_GOOGLE_COLAB:

    GENERATED_MODEL_PATH = GOOGLE_COLAB_ROOT + GENERATED_MODEL_PATH
    DATA_NOTES_DIR = GOOGLE_COLAB_ROOT + DATA_NOTES_DIR

def generate(filename):
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open(DATA_NOTES_DIR, 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    # model = create_network(normalized_input, n_vocab)
    model = load_model(GENERATED_MODEL_PATH)

    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output, filename)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    # output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        # output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(GENERATED_MODEL_PATH)

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1) #randomowa tablica intów

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames)) #int -> nuta dict

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output, file_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=file_path)


In [ ]:

if __name__ == '__main__':
    generate()
